##### Experiment, sheet 11:
Tot nu toe zie ik 3 manieren om tekst+tekstcoordinaten te implementeren: 1. op basis van een onveranderlijke lijst van basissegmenten (zoals de lines in Republic), 2. op basis van een initiele segmentatie die wordt bepaald door hoe de te verwerken bron al is opgedeeld (zoals het TEI voorbeeld, met head, section, paragraph en page), waarbij de segmentatie later is te verfijnen, 3. op basis van een initiele segmentatie, zoals in geval 2, waarbij fijnmaziger segmentatie kan worden toegevoegd op basis van relatieve char offset tov het initiele segment in kwestie. 
Ik heb cases 1 en 2 inmiddels geimplementeerd als subclasses van een abstract class SegmentedText. Case 2 werkt (per definitie) al, want identiek aan sheet 10.
In deze sheet pas ik de recentste Republic code uit sheet 7 aan, om gebruik te maken van IndexedSegmentedText, in plaats van van een simpele list. IndexedSegmentedText doet overigens niet veel meer dan een wrapper voor een list zijn.

Mogelijk vervolg:
- zoek uit of ik ook een abstracte Anchor class nodig heb
- instantieer voor beide cases de juiste SegmentedText en gebruik die in de REST services

In [1]:
import sys
sys.path.append('../packages')

from textservice import segmentedtext
from annotation import asearch

datadir = '../data/'
resource_id = 'volume-1705-1'
text_store = 'textstore.json'
annotation_store = 'annotationstore.json'

In [2]:
import json
import glob
import re

# read files

all_textlines=segmentedtext.IndexedSegmentedText(resource_id)
all_annotations=[]

# We want to load 'text containers' that contain more or less contiguous text and are as long as practically
# possible. Container size is determined by pragmatic reasons, e.g. technical (performance) or user driven
# (corresponding with all scans in a book or volume). This functions returns all component files IN TEXT ORDER.
# Examples: sorted list of files, part of IIIF manifest.

def get_file_sequence_for_container(text_container):
    path = "../data/sessions/meeting-1705*"
    session_file_names = (f for f in glob.glob(path))
    return sorted(session_file_names)

# Many file types contain a hierarchy of ordered text and/or annotation elements of different types. Some form of
# depth-first, post order traversal is necessary. Examples: processing a json hierarchy with dictionaries
# and lists (republic) or parsing TEI XML (DBNL document).

def traverse(node,node_label,text,annotations):
    # find the list that represents the children, each child is a dict, assume first list is the correct one
    children = []
    label_of_children = ''
    for key,val in node.items():
        if (type(val) == list):
            children = val
            label_of_children = key
            break 
    
    if 'coords' in node:
        coords = node['coords']
    else:
        coords = None
    
    begin_index = text.len()
    annotation_info = {'resource_id': resource_id, 'label' : node_label,'image_coords': coords,'begin_anchor' : begin_index}
    if len(children) == 0:        # if no children, do your 'leaf node thing'
        annotation_info['id'] = node['id']
        annotation_info['end_anchor'] = text.len()
        node_text = node['text']
        
        if node_text is None:
            node_text = '\n'

        text.append(node_text)
    else:                         # if non-leaf node, first visit children     
        for child in children:
            traverse(child,label_of_children,text,annotations)
        
        # ONDERSTAANDE IS SMERIG, hangt van onzekere aannames af
        for k in node['metadata'].keys():
            idkey = ''
            if k.endswith('id'):
                idkey = k
                break
        annotation_info['id'] = node['metadata'][idkey]
        
        end_index = text.len()-1
        annotation_info['end_anchor'] = end_index    # after child text segments are added
        
        # if node contains iiif_url, create extra annotation_info for 'scanpage'
        if 'iiif_url' in node['metadata']:
            scan_annot_info = {'resource_id': resource_id, 'label':'scanpage','iiif_url':node['metadata']['iiif_url'],\
                               'begin_anchor':begin_index,'end_anchor':end_index}
            scan_annot_info['scan_num'] = node['metadata']['scan_num']
            annotations.append(scan_annot_info)
        
    annotations.append(annotation_info)
    return

# In case of presence of a hierarchical structure, processing/traversal typically starts from a root element.

def get_root_tree_element(file):
    with open(file, 'r') as myfile:
        session_file=myfile.read() 
        
    session_data = json.loads(session_file)      
    return session_data['_source']

# Rudimentary version of a scanpage_handler

def deduplicate_scanpage_annotations(a_array):
    # use a generator to create a list of only scanpage annotation_info dicts
    scan_page_annots = [ann_info for ann_info in a_array if ann_info['label'] == 'scanpage']
        
    # use groupBy on a list of dicts (zie Python cookbook 1.15)
    from operator import itemgetter
    from itertools import groupby

    # first sort on scan_num
    scan_page_annots.sort(key=itemgetter('scan_num'))

    # iterate in groups
    aggregated_scan_annots = []

    for scan_num, items in groupby(scan_page_annots, key=itemgetter('scan_num')):
        # first, convert the 'items' iterator to a list, to able to use it twice (iterators can be used once)
        itemlist = list(items)
    
        # copy the item with the lowest begin_index
        aggr_scan_annot = min(itemlist, key=itemgetter('begin_anchor')).copy()
    
        # replace 'end_anchor' with the highest end_index in the group
        max_end_index = max(itemlist, key=itemgetter('end_anchor'))['end_anchor']
        aggr_scan_annot['end_anchor'] = max_end_index
        
        # add to result
        aggregated_scan_annots.append(aggr_scan_annot)

#    for scan_ann in aggregated_scan_annots:
#        scan_ann['iiif_url'] = re.sub(r'(\d+),(\d+),(\d+),(\d+)/(full)', r'\5/,\4', scan_ann['iiif_url'])
    
    a_array = [ann for ann in a_array if ann not in scan_page_annots]
    a_array.extend(aggregated_scan_annots)
           
    return

def correct_scanpage_imageurls(a_array):
    scan_page_annots = [ann_info for ann_info in a_array if ann_info['label'] == 'scanpage']
    
    for scan_ann in scan_page_annots:
        scan_ann['iiif_url'] = re.sub(r'(\d+),(\d+),(\d+),(\d+)/(full)', r'\5/,\4', scan_ann['iiif_url'])
        
    return

# Rudimentary version of a page_handler

def add_page_annotations(source_data, ann_array):
    page_data = source_data['page_versions']
        
    # generator
    page_identifiers = (pg['page_id'] for pg in page_data)
    page_annots = [{'resource_id': resource_id, 'label' : 'pages','id' : page_id} for page_id in page_identifiers]
        
    for pa in page_annots:
        scan_num = int(re.search(r'(\d+)-page-', pa['id']).group(1))
        scanpage_for_scan_num = [ai for ai in annotation_array if 'scan_num' in ai.keys() and ai['scan_num'] == \
                                 scan_num]
        pa['begin_anchor'] = scanpage_for_scan_num[0]['begin_anchor']
        pa['end_anchor'] = scanpage_for_scan_num[0]['end_anchor']
        pa['indexesByContainment'] = True
     
    ann_array.extend(page_annots)
    return

# Process per file, properly concatenate results, maintaining proper referencing the baseline text elements
for f_name in get_file_sequence_for_container(resource_id):
    text_array = segmentedtext.IndexedSegmentedText()
    annotation_array = []
            
    source_data = get_root_tree_element(f_name)

    traverse(source_data,'sessions',text_array,annotation_array)
        
    scanpages = deduplicate_scanpage_annotations(annotation_array) 
    correct_scanpage_imageurls(annotation_array)
    
    add_page_annotations(source_data, annotation_array)
           
    # properly concatenate annotation info taking ongoing line indexes into account
    for ai in annotation_array:
        ai['begin_anchor'] += all_textlines.len()
        ai['end_anchor'] += all_textlines.len()
    
    all_textlines.extend(text_array)       
    all_annotations.extend(annotation_array)

Lees resolutie info in uit de resultaten van queries naar Marijn's resolutie-index, voor de betreffende 7 zittingsdagen. Vooralsnog neem ik alleen resolutie id en line-ids van de eerste en laatste regel van de resolutie mee.

In [3]:
resolution_annotations=[]

def get_resolution_files_for_container(text_container):
    path = "../data/resolutions/*-resolutions.json"
    resolution_file_names = (f for f in glob.glob(path))
    return sorted(resolution_file_names)

def res_traverse(node, line_ids):
    # find the list that represents the children, each child is a dict, assume first list is the correct one
    children = []
    label_of_children = ''
    
    # assume, first list in dict are the children
    for key,val in node.items():
        # HACK ALERT! assumption that first list contains children has exceptions
        if type(val) == list and key != 'paragraphs' and key != 'evidence':
            children = val
            label_of_children = key
            break 
    
    if len(children) == 0:        # if no children, do your 'leaf node thing'
        line_ids.append(node['metadata']['id'])
    else:                         # if non-leaf node, first visit children     
        for child in children:
            res_traverse(child,line_ids)
                            
    return

# In case of presence of a hierarchical structure, processing/traversal typically starts from a root element.

def get_res_root_element(file):
    with open(file, 'r') as myfile:
        resolution_file=myfile.read() 
        
    resolution_data = json.loads(resolution_file)      
    return resolution_data['hits']['hits']

for f_name in get_resolution_files_for_container(resource_id):    
    # get list of resolution 'hits'
    hits = get_res_root_element(f_name)
    print(f_name)
    for hit in hits:
        # each hit corresponds with a resolution
        resolution_line_ids = []
        res_traverse(hit['_source'],resolution_line_ids)
        
        resolution_info = {'resource_id': resource_id, 'label' : 'resolutions','begin_anchor' : resolution_line_ids[0], \
                                      'end_anchor': resolution_line_ids[len(resolution_line_ids)-1], 'id': hit['_id']}
        resolution_annotations.append(resolution_info)

resolution_annotations

../data/resolutions/1705-01-02-resolutions.json
../data/resolutions/1705-01-03-resolutions.json
../data/resolutions/1705-01-06-resolutions.json
../data/resolutions/1705-01-07-resolutions.json
../data/resolutions/1705-01-09-resolutions.json
../data/resolutions/1705-01-10-resolutions.json
../data/resolutions/1705-01-11-resolutions.json


[{'resource_id': 'volume-1705-1',
  'label': 'resolutions',
  'begin_anchor': 'NL-HaNA_1.01.02_3760_0011-page-20-column-0-tr-0-line-26',
  'end_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-0-tr-0-line-7',
  'id': 'meeting-1705-01-02-session-1-resolution-27'},
 {'resource_id': 'volume-1705-1',
  'label': 'resolutions',
  'begin_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-0-tr-0-line-8',
  'end_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-0-tr-0-line-20',
  'id': 'meeting-1705-01-02-session-1-resolution-28'},
 {'resource_id': 'volume-1705-1',
  'label': 'resolutions',
  'begin_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-0-tr-0-line-21',
  'end_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-0-tr-0-line-43',
  'id': 'meeting-1705-01-02-session-1-resolution-29'},
 {'resource_id': 'volume-1705-1',
  'label': 'resolutions',
  'begin_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-0-tr-0-line-44',
  'end_anchor': 'NL-HaNA_1.01.02_3760_0012-page-22-column-1-tr-0

Maak voor alle line-ids een dict aan van line-id versus text_index

In [4]:
line_ids_vs_indexes = {}
for line in all_annotations:
    if line['label'] == 'lines':
        line_ids_vs_indexes.update({line['id'] : line['begin_anchor']})
        
line_ids_vs_indexes

{'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-8': 0,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-9': 1,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-10': 2,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-11': 3,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-12': 4,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-13': 5,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-14': 6,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-0-line-15': 7,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-4-line-2': 8,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-4-line-3': 9,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-1-line-0': 10,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-1-line-1': 11,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-1-line-2': 12,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-1-line-3': 13,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-1-line-4': 14,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr-1-line-5': 15,
 'NL-HaNA_1.01.02_3760_0008-page-15-col-0-tr

Voor alle resoluties, voeg begin_index en end_index toe.
Opmerking: marijn gebruikt bij zittingsdagen 'col' in de ids, en bij resoluties 'column'. Fix mbv regex.

In [5]:
for res in resolution_annotations:
    num_errors = 0
    try:
        res['begin_anchor'] = re.sub(r'-column-', r'-col-', res['begin_anchor'])
        res['end_anchor'] = re.sub(r'-column-', r'-col-', res['end_anchor'])
    
        res['begin_anchor'] = line_ids_vs_indexes[res['begin_anchor']]
        res['end_anchor'] = line_ids_vs_indexes[res['end_anchor']]
    except:
        res['begin_anchor'] = 0
        res['end_anchor'] = 0
        num_errors += 1
    if num_errors > 0:
        print(f"number of lookup errors: {num_errors}")
    
resolution_annotations

number of lookup errors: 1


[{'resource_id': 'volume-1705-1',
  'label': 'resolutions',
  'begin_anchor': 599,
  'end_anchor': 830,
  'id': 'meeting-1705-01-02-session-1-resolution-27'},
 {'resource_id': 'volume-1705-1',
  'label': 'resolutions',
  'begin_anchor': 831,
  'end_anchor': 843,
  'id': 'meeting-1705-01-02-session-1-resolution-28'},
 {'resource_id': 'volume-1705-1',
  'label': 'resolutions',
  'begin_anchor': 844,
  'end_anchor': 866,
  'id': 'meeting-1705-01-02-session-1-resolution-29'},
 {'resource_id': 'volume-1705-1',
  'label': 'resolutions',
  'begin_anchor': 867,
  'end_anchor': 885,
  'id': 'meeting-1705-01-02-session-1-resolution-30'},
 {'resource_id': 'volume-1705-1',
  'label': 'resolutions',
  'begin_anchor': 886,
  'end_anchor': 899,
  'id': 'meeting-1705-01-02-session-1-resolution-31'},
 {'resource_id': 'volume-1705-1',
  'label': 'resolutions',
  'begin_anchor': 901,
  'end_anchor': 945,
  'id': 'meeting-1705-01-02-session-1-resolution-32'},
 {'resource_id': 'volume-1705-1',
  'label': '

In [6]:
all_annotations.extend(resolution_annotations)

Volgende stap: definieren van named entities. Voorlopig alleen door te verwijzen naar de index van de line(s) waar deze entities op voorkomen

In [7]:
entity_segments = [(2814,2814,'per','Arnoldus Boomhouwer'),(2832,2833,'per','van Rabenpeé'),\
                   (2819,2819,'loc','Roermond')]

id_suffix = 1
entity_annotations = []
for es in entity_segments:
    ent_annot = {'resource_id': resource_id, 'label':'entities','begin_anchor':es[0],'end_anchor':es[1],\
                 'id':'NL-HaNA_1.01.02_3760_entity-'+str(id_suffix),\
                 'entity_type': es[2],'entity_text':es[3]}
    entity_annotations.append(ent_annot)
    id_suffix += 1

entity_annotations

[{'resource_id': 'volume-1705-1',
  'label': 'entities',
  'begin_anchor': 2814,
  'end_anchor': 2814,
  'id': 'NL-HaNA_1.01.02_3760_entity-1',
  'entity_type': 'per',
  'entity_text': 'Arnoldus Boomhouwer'},
 {'resource_id': 'volume-1705-1',
  'label': 'entities',
  'begin_anchor': 2832,
  'end_anchor': 2833,
  'id': 'NL-HaNA_1.01.02_3760_entity-2',
  'entity_type': 'per',
  'entity_text': 'van Rabenpeé'},
 {'resource_id': 'volume-1705-1',
  'label': 'entities',
  'begin_anchor': 2819,
  'end_anchor': 2819,
  'id': 'NL-HaNA_1.01.02_3760_entity-3',
  'entity_type': 'loc',
  'entity_text': 'Roermond'}]

In [8]:
all_annotations.extend(entity_annotations)

In [9]:
# voor iedere annotatie, bepaal image_range en voeg deze toe

def get_bounding_box_for(annotations): 
    ann_list = list(annotations) # because a generator can only be used once
    
    min_left = min([ann['image_coords']['left'] for ann in ann_list if 'image_coords' in ann])
    max_right = max([ann['image_coords']['right'] for ann in ann_list if 'image_coords' in ann])
    min_top = min([ann['image_coords']['top'] for ann in ann_list if 'image_coords' in ann])
    max_bottom = max([ann['image_coords']['bottom'] for ann in ann_list if 'image_coords' in ann])
    height = max_bottom - min_top
    width = max_right - min_left

    return {'left': min_left, 'right': max_right, 'top': min_top, 'bottom': max_bottom, 'height': height, 'width': width}

def add_image_range(ann):
    ann['image_range'] = []
    
    ann_begin=ann['begin_anchor']
    ann_end=ann['end_anchor']
        
    # loop over scans die overlappen met de annotatie
    for a in asearch.get_annotations_of_type_overlapping('scanpage',ann_begin,ann_end,all_annotations):
        bounding_boxes = []
        image_url = a['iiif_url']
        
        scan_begin=a['begin_anchor']
        scan_end=a['end_anchor']
        
        # loop over alle kolommen op de betreffende scan. Per kolom, bereken bounding box voor 
        # overlappende resolutieregels
        for clm in asearch.get_annotations_of_type_overlapping('columns',scan_begin,scan_end,all_annotations):
            clm_begin=clm['begin_anchor']
            clm_end=clm['end_anchor']
            
            # bepaal overlap_begin en overlap_end indexes voor kolom
            overlap_begin=max(ann_begin, clm_begin)
            overlap_end=min(ann_end, clm_end)
                        
            # bepaal hieruit de bounding box coords voor deze kolom
            if overlap_end-overlap_begin >= 0: # resolution and column are overlapping
                bounding_box=get_bounding_box_for(asearch.get_annotations_of_type_overlapping('lines',\
                                                        overlap_begin,overlap_end,all_annotations))
                bounding_boxes.append(bounding_box)
        
        ann['image_range'].append((image_url, bounding_boxes))
        print(ann['image_range'])
    return            

In [10]:
def add_region_links(ann):
    region_links = []
    try:
        for image_url, regions in ann['image_range']:
            for coords in regions:
                # construct iiif_url from image_url and coords
                coord_str = f"{coords['left']},{coords['top']},{coords['width']},{coords['height']}"
                region_url = re.sub(r'(full)/(,\d+)', rf'{coord_str}/\1', image_url)
                region_links.append(region_url)
    except:
        print('error: annotation without image range')
        
    ann['region_links'] = region_links
    return

In [11]:
for a in all_annotations:
    add_image_range(a)  
    add_region_links(a)

[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/full/,1616/0/default.jpg', [{'left': 2691, 'right': 3396, 'top': 1850, 'bottom': 1921, 'height': 71, 'width': 705}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/full/,1616/0/default.jpg', [{'left': 2817, 'right': 3098, 'top': 1899, 'bottom': 1957, 'height': 58, 'width': 281}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/full/,1616/0/default.jpg', [{'left': 3188, 'right': 3388, 'top': 1910, 'bottom': 1966, 'height': 56, 'width': 200}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/full/,1616/0/default.jpg', [{'left': 3237, 'right': 3379, 'top': 1954, 'bottom': 2010, 'height': 56, 'width': 142}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/full/,1616/0/default.jpg', [{'left': 2680, 'right': 3238, 'top': 1993, 'bottom': 2058, 'heigh

[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/full/,1116/0/default.jpg', [{'left': 4094, 'right': 4381, 'top': 912, 'bottom': 976, 'height': 64, 'width': 287}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/full/,1116/0/default.jpg', [{'left': 4090, 'right': 4374, 'top': 964, 'bottom': 1023, 'height': 59, 'width': 284}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/full/,1116/0/default.jpg', [{'left': 4165, 'right': 4379, 'top': 1014, 'bottom': 1070, 'height': 56, 'width': 214}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/full/,1116/0/default.jpg', [{'left': 4090, 'right': 4381, 'top': 826, 'bottom': 1070, 'height': 244, 'width': 291}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0008.jpg/full/,1116/0/default.jpg', [{'left': 3965, 'right': 4360, 'top': 1301, 'bottom': 1366, 'height':

[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3114/0/default.jpg', [{'left': 428, 'right': 1307, 'top': 3216, 'bottom': 3273, 'height': 57, 'width': 879}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3114/0/default.jpg', [{'left': 399, 'right': 1317, 'top': 359, 'bottom': 3273, 'height': 2914, 'width': 918}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3114/0/default.jpg', [{'left': 399, 'right': 1317, 'top': 359, 'bottom': 3273, 'height': 2914, 'width': 918}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3114/0/default.jpg', [{'left': 399, 'right': 1317, 'top': 359, 'bottom': 3273, 'height': 2914, 'width': 918}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3119/0/default.jpg', [{'left': 1367, 'right': 2256, 'top': 349, 'bottom': 406, 'height':

[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3125/0/default.jpg', [{'left': 2544, 'right': 3429, 'top': 625, 'bottom': 691, 'height': 66, 'width': 885}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3125/0/default.jpg', [{'left': 2541, 'right': 3427, 'top': 675, 'bottom': 743, 'height': 68, 'width': 886}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3125/0/default.jpg', [{'left': 2544, 'right': 3117, 'top': 724, 'bottom': 783, 'height': 59, 'width': 573}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3125/0/default.jpg', [{'left': 3130, 'right': 3297, 'top': 733, 'bottom': 793, 'height': 60, 'width': 167}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3125/0/default.jpg', [{'left': 2576, 'right': 3427, 'top': 772, 'bottom': 835, 'height': 63, 'w

[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3064/0/default.jpg', [{'left': 3507, 'right': 4380, 'top': 1552, 'bottom': 1622, 'height': 70, 'width': 873}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3064/0/default.jpg', [{'left': 3507, 'right': 4381, 'top': 1601, 'bottom': 1670, 'height': 69, 'width': 874}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3064/0/default.jpg', [{'left': 3502, 'right': 4386, 'top': 1648, 'bottom': 1720, 'height': 72, 'width': 884}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3064/0/default.jpg', [{'left': 3504, 'right': 4381, 'top': 1697, 'bottom': 1767, 'height': 70, 'width': 877}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0009.jpg/full/,3064/0/default.jpg', [{'left': 3504, 'right': 4385, 'top': 1746, 'bottom': 1817, 'heigh

...

[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,3116/0/default.jpg', [{'left': 480, 'right': 1320, 'top': 1268, 'bottom': 1325, 'height': 57, 'width': 840}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,3116/0/default.jpg', [{'left': 428, 'right': 1314, 'top': 1313, 'bottom': 1376, 'height': 63, 'width': 886}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,3116/0/default.jpg', [{'left': 434, 'right': 1320, 'top': 1365, 'bottom': 1422, 'height': 57, 'width': 886}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,3116/0/default.jpg', [{'left': 436, 'right': 1320, 'top': 1414, 'bottom': 1473, 'height': 59, 'width': 884}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,3116/0/default.jpg', [{'left': 435, 'right': 1320, 'top': 1462, 'bottom': 1520, 'height': 5

[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,3116/0/default.jpg', [{'left': 1409, 'right': 2303, 'top': 2476, 'bottom': 2535, 'height': 59, 'width': 894}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,3116/0/default.jpg', [{'left': 1410, 'right': 2298, 'top': 2525, 'bottom': 2582, 'height': 57, 'width': 888}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,3116/0/default.jpg', [{'left': 1412, 'right': 2293, 'top': 2573, 'bottom': 2632, 'height': 59, 'width': 881}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,3116/0/default.jpg', [{'left': 1415, 'right': 2303, 'top': 2621, 'bottom': 2680, 'height': 59, 'width': 888}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,3116/0/default.jpg', [{'left': 1414, 'right': 2297, 'top': 2670, 'bottom': 2729, 'heigh

[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,1582/0/default.jpg', [{'left': 2591, 'right': 3313, 'top': 2226, 'bottom': 2302, 'height': 76, 'width': 722}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,1582/0/default.jpg', [{'left': 2586, 'right': 3416, 'top': 2274, 'bottom': 2351, 'height': 77, 'width': 830}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,1582/0/default.jpg', [{'left': 2636, 'right': 3247, 'top': 2324, 'bottom': 2396, 'height': 72, 'width': 611}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,1582/0/default.jpg', [{'left': 2590, 'right': 3417, 'top': 2371, 'bottom': 2449, 'height': 78, 'width': 827}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0032.jpg/full/,1582/0/default.jpg', [{'left': 2637, 'right': 2886, 'top': 2422, 'bottom': 2481, 'heigh

[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0013.jpg/full/,3122/0/default.jpg', [{'left': 398, 'right': 1290, 'top': 1186, 'bottom': 1731, 'height': 545, 'width': 892}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0013.jpg/full/,3122/0/default.jpg', [{'left': 404, 'right': 1291, 'top': 1720, 'bottom': 2115, 'height': 395, 'width': 887}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0013.jpg/full/,3122/0/default.jpg', [{'left': 405, 'right': 1295, 'top': 2108, 'bottom': 2418, 'height': 310, 'width': 890}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0013.jpg/full/,3122/0/default.jpg', [{'left': 409, 'right': 1299, 'top': 2401, 'bottom': 2954, 'height': 553, 'width': 890}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0013.jpg/full/,3122/0/default.jpg', [{'left': 408, 'right': 1299, 'top': 2936, 'bottom': 3281, 'height

[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0021.jpg/full/,3129/0/default.jpg', [{'left': 2406, 'right': 3294, 'top': 1079, 'bottom': 2443, 'height': 1364, 'width': 888}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0021.jpg/full/,3129/0/default.jpg', [{'left': 2411, 'right': 3284, 'top': 2437, 'bottom': 3283, 'height': 846, 'width': 873}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0021.jpg/full/,3129/0/default.jpg', [{'left': 2411, 'right': 3284, 'top': 2437, 'bottom': 3283, 'height': 846, 'width': 873}]), ('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0021.jpg/full/,1734/0/default.jpg', [{'left': 3257, 'right': 4277, 'top': 292, 'bottom': 1098, 'height': 806, 'width': 1020}])]
[('https://images.diginfra.net/iiif/NL-HaNA_1.01.02/3760/NL-HaNA_1.01.02_3760_0021.jpg/full/,1734/0/default.jpg', [{'left': 3363, 'right': 4253, 'top': 1084, 'bottom': 1826, 'h

In [12]:
def add_segmented_text_to_store(segmented_text, store_name):
    try:
        with open(datadir+store_name, 'r') as filehandle:
            data = json.loads(filehandle.read())
    except FileNotFoundError:
        data = {'_resources' : []}
        
    data['_resources'].append(segmented_text)
    
    with open(datadir+store_name, 'w') as filehandle:
        json.dump(data, filehandle, cls=segmentedtext.SegmentEncoder)     

In [13]:
add_segmented_text_to_store(all_textlines, text_store)

In [14]:
def add_annotations_to_store(annotations, store_name):
    try:
        with open(datadir+store_name, 'r') as filehandle:
            data = json.loads(filehandle.read())
    except FileNotFoundError:
        data = []
        
    data.extend(annotations)
    
    with open(datadir+store_name, 'w') as filehandle:
        json.dump(data, filehandle, cls=segmentedtext.AnchorEncoder)    

In [15]:
add_annotations_to_store(all_annotations, annotation_store)